In [1]:
import h5py
import numpy as np
from numpy.random import default_rng
from sklearn.metrics import accuracy_score

from smml.kernels import GaussianKernel
from smml.models import Pegasos

d:\Users\gabri\anaconda3\envs\p3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PATH = '../../datasets/usps/usps.h5'
with h5py.File(PATH, 'r') as hf:
        train = hf.get('train')
        X_tr = train.get('data')[:]
        y_tr = train.get('target')[:]
        test = hf.get('test')
        X_te = test.get('data')[:]
        y_te = test.get('target')[:]

In [3]:
peg = Pegasos()

In [4]:
y = np.where(y_tr == 8, 1, -1)

In [5]:
peg.fit(X_tr, y)

100%|██████████| 1000/1000 [00:00<00:00, 48760.20it/s]


In [6]:
peg.predict_proba(X_te)

array([-1.46142509e-09, -1.19160688e-25, -1.38055209e-13, ...,
       -7.63361873e-28, -3.22004553e-17, -4.94595906e-04])

In [ ]:
class MulticlassPegasos:
    pass
    # should train multiple pegasos etc

In [ ]:
y_tr

In [ ]:
y_tr_0 = np.where(y_tr == 0, 1, -1)

In [ ]:
pegasos = Pegasos()

In [ ]:
pegasos.fit(X_tr, y_tr_0)

In [ ]:
y_pred = pegasos.predict(X_te)

In [ ]:
y_pred

In [ ]:
y_te_0 = np.where(y_te == 0, 1, -1)

In [ ]:
accuracy_score(y_pred, y_te_0)